In [1]:
import pandas as pd
import numpy as np
import gensim
import requests
from bs4 import BeautifulSoup
import time
import re

In [42]:
cities = pd.read_csv('./datasets/cities_clean.csv')
cities.head()

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link
0,0,"Paradise, CA",1066843491555205120,"Lord Farquad Quad Squat Squad @Chico, Californ...",2018-11-25 23:58:04+00:00,NaN,areoandmilk,NaN,0,0,https://twitter.com/areoandmilk/status/1066843...
1,1,"Paradise, CA",1066842521601400832,winter edition #queenadailypic released. @Soda...,2018-11-25 23:54:12+00:00,#queenadailypic,leenathequeena,NaN,0,0,https://twitter.com/leenathequeena/status/1066...
2,2,"Paradise, CA",1066841740060098562,Drinking a Def Leppard Pale by @ElysianBrewing...,2018-11-25 23:51:06+00:00,NaN,geradellsworth,@ElysianBrewing @Golden1Center,0,0,https://twitter.com/geradellsworth/status/1066...
3,3,"Paradise, CA",1066841178782482433,"The forgotten. #LimeBike #Reno @Reno, Nevada h...",2018-11-25 23:48:52+00:00,#LimeBike #Reno,alittlegordie,NaN,0,0,https://twitter.com/alittlegordie/status/10668...
4,4,"Paradise, CA",1066839617196961792,SSCC is United and moving Forward !!! The powe...,2018-11-25 23:42:40+00:00,#heritageoffaith,LesSimmons,@sscc7710,1,0,https://twitter.com/LesSimmons/status/10668396...


In [3]:
# counts of hashtages (strip the #)
def count_hashtags(df):
    keyterms = {}
    for i in df['hashtags'].dropna():
        i = i.replace("#", '')
        for j in i.split():
            if j not in keyterms.keys():
                keyterms[j] = 1
            else:
                keyterms[j] += 1
    return keyterms

In [12]:
def build_keywords(hashtags): 
    word_filter = ['fire', 'evac', 'smok', 'burn', 'wild', 'blaz', 'hell', 'department',
              'inferno', 'help']
    keywords = []
    for word in set(hashtags):
        for wf in word_filter:
            if wf in word.lower():
                keywords.append(word)
    return list(set(keywords))

In [8]:
# check each tweet for keyterms and score them
def keyscoring(df, keywords):
    keyscore = []
    for row in df.index:
        keyscore.append(0)
        for word in str(df.iloc[row]['tweet_text']).split():
            if word.lower().replace('#','') in keywords:
                try:
                    keyscore[row]+=1
                except:
                    print(f'fail in row {row}')
    df['key_score'] = keyscore


In [44]:
def get_locs(df):
    locs = []
    df_num_rows = df.shape[0]
    for row in df.index:
        url = df.iloc[row]['link']
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        spans = soup.find_all('span', {'class' : 'permalink-tweet-geo-text'})
        try:
            location = spans[0].text.replace("from",'').strip()
            if location == 'California, USA':
                locs.append(df.iloc[row]['City'])
            else:
                locs.append(location)
        except:
            print(f'tweet #{row} has no location info')
            locs.append('nolocationfound')
        
        if row % 25 == 0:
            time.sleep(3)
        if row % 75 == 0:
            print(f'Located {row} out of {df_num_rows} tweets.  {row/df_num_rows:.2%}')
    df['from_locations'] = locs


In [4]:
def remove_url(df):
    df['text_nourl'] = [re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', i) for i in df['tweet_text']]

In [38]:
def func_master(df):
    keywords = build_keywords(list(count_hashtags(df)))
    remove_url(df)
    keyscoring(df, keywords)
    get_locs(df)

In [45]:
func_master(cities)
cities.head()

Located 0 out of 1809 tweets.  0.00%
Located 75 out of 1809 tweets.  4.15%
Located 150 out of 1809 tweets.  8.29%
Located 225 out of 1809 tweets.  12.44%
Located 300 out of 1809 tweets.  16.58%
tweet #341 has no location info
Located 375 out of 1809 tweets.  20.73%
Located 450 out of 1809 tweets.  24.88%
Located 525 out of 1809 tweets.  29.02%
Located 600 out of 1809 tweets.  33.17%
Located 675 out of 1809 tweets.  37.31%
Located 750 out of 1809 tweets.  41.46%
tweet #801 has no location info
Located 825 out of 1809 tweets.  45.61%
Located 900 out of 1809 tweets.  49.75%
Located 975 out of 1809 tweets.  53.90%
Located 1050 out of 1809 tweets.  58.04%
Located 1125 out of 1809 tweets.  62.19%
Located 1200 out of 1809 tweets.  66.33%
Located 1275 out of 1809 tweets.  70.48%
Located 1350 out of 1809 tweets.  74.63%
Located 1425 out of 1809 tweets.  78.77%
Located 1500 out of 1809 tweets.  82.92%
Located 1575 out of 1809 tweets.  87.06%
Located 1650 out of 1809 tweets.  91.21%
Located 1725 

,tweet_count,City,id,tweet_text,timestamp,hashtags,username,mentions,rewtweets,replies,link,text_nourl,key_score,from_locations
0,0,"Paradise, CA",1066843491555205120,"Lord Farquad Quad Squat Squad @Chico, Californ...",2018-11-25 23:58:04+00:00,NaN,areoandmilk,NaN,0,0,https://twitter.com/areoandmilk/status/1066843...,"Lord Farquad Quad Squat Squad @Chico, California",0,"Chico, CA"
1,1,"Paradise, CA",1066842521601400832,winter edition #queenadailypic released. @Soda...,2018-11-25 23:54:12+00:00,#queenadailypic,leenathequeena,NaN,0,0,https://twitter.com/leenathequeena/status/1066...,winter edition #queenadailypic released. @Soda...,0,"Paradise, CA"
2,2,"Paradise, CA",1066841740060098562,Drinking a Def Leppard Pale by @ElysianBrewing...,2018-11-25 23:51:06+00:00,NaN,geradellsworth,@ElysianBrewing @Golden1Center,0,0,https://twitter.com/geradellsworth/status/1066...,Drinking a Def Leppard Pale by @ElysianBrewing...,0,"Sacramento, CA"
3,3,"Paradise, CA",1066841178782482433,"The forgotten. #LimeBike #Reno @Reno, Nevada h...",2018-11-25 23:48:52+00:00,#LimeBike #Reno,alittlegordie,NaN,0,0,https://twitter.com/alittlegordie/status/10668...,"The forgotten. #LimeBike #Reno @Reno, Nevada",0,"Reno, NV"
4,4,"Paradise, CA",1066839617196961792,SSCC is United and moving Forward !!! The powe...,2018-11-25 23:42:40+00:00,#heritageoffaith,LesSimmons,@sscc7710,1,0,https://twitter.com/LesSimmons/status/10668396...,SSCC is United and moving Forward !!! The powe...,0,"Sacramento, CA"


In [10]:
hashtags = list(count_hashtags(cities))

keywords = build_keywords(hashtags)

remove_url(cities)

keyscoring(cities, keywords)
cities['key_score'].value_counts()

In [17]:
get_locs(cities)
cities['from_locations'].value_counts()

tweet #1 has no location info
tweet #28 has no location info
tweet #33 has no location info
tweet #52 has no location info
tweet #54 has no location info
tweet #63 has no location info
tweet #73 has no location info
Located 75 out of 1809 tweets.  4.15%
tweet #84 has no location info
tweet #100 has no location info
tweet #129 has no location info
tweet #148 has no location info
Located 150 out of 1809 tweets.  8.29%
tweet #154 has no location info
tweet #164 has no location info
tweet #186 has no location info
tweet #195 has no location info
tweet #215 has no location info
tweet #217 has no location info
Located 225 out of 1809 tweets.  12.44%
tweet #240 has no location info
tweet #244 has no location info
tweet #267 has no location info
tweet #295 has no location info
Located 300 out of 1809 tweets.  16.58%
tweet #315 has no location info
tweet #331 has no location info
tweet #334 has no location info
tweet #341 has no location info
tweet #345 has no location info
tweet #347 has no lo

tweet #1637 has no location info
tweet #1639 has no location info
Located 1650 out of 1809 tweets.  91.21%
tweet #1689 has no location info
tweet #1690 has no location info
tweet #1705 has no location info
tweet #1706 has no location info
tweet #1712 has no location info
Located 1725 out of 1809 tweets.  95.36%
tweet #1725 has no location info
tweet #1727 has no location info
tweet #1729 has no location info
tweet #1739 has no location info
tweet #1758 has no location info
tweet #1768 has no location info
tweet #1773 has no location info
tweet #1789 has no location info
tweet #1793 has no location info
tweet #1795 has no location info
tweet #1798 has no location info
tweet #1799 has no location info
Located 1800 out of 1809 tweets.  99.50%
tweet #1800 has no location info
tweet #1801 has no location info
tweet #1806 has no location info


Chico, CA             791
Paradise, CA          576
nolocationfound       250
Oroville, CA          112
Magalia, CA            17
Durham, CA             12
Gridley, CA            12
South Oroville, CA      9
Thermalito, CA          8
Sacramento, CA          8
Oroville East, CA       3
Folsom, CA              2
Glen Ellen, CA          1
Reno, NV                1
Yountville, CA          1
Napa, CA                1
Linda, CA               1
Nevada City, CA         1
Redding, CA             1
Rocklin, CA             1
Florin, CA              1
Name: from_locations, dtype: int64

In [59]:
#  pd.set_option('display.max_rows', None)

### Currently unused

In [7]:
# use hashtag list to build dictionary of keyterms to use combined with that of the gensim model
# Import word vectors into "model."
model = gensim.models.KeyedVectors.load_word2vec_format('../lexvec.enwiki+newscrawl.300d.W.pos.vectors')


In [8]:
keyt = []
keywords = ['fire', 'smoke', 'wildfire', 'campfire', 'forest', 'evacuate', 'hell']
for word in keywords:
    kt = [x[0] for x in model.most_similar(word, topn = 25)]
    keyt+= kt
word_list = list(set(keyt))

In [9]:
important_words = []
for word in keyterms.keys():
    if word in word_list:
        important_words.append(word)